In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

plt.figure(figsize=(16,6))
%matplotlib inline

file_location = "my_folder_location/"

In [ ]:
# Changing column types crib-sheet

df["col"] = pd.to_datetime(df["col"], format = "%d/%m/%y, %H:%M")
# Format strings: https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior 

cat_cols = ["A","B","C"]
for col in cat_cols:
    df[col] = df[col].astype("category")

In [ ]:
# Return a string from a dataframe
df.loc[df["col"]==value,"col_to_return"].values.item()

In [ ]:
# A stacked bar chart.
new_esol["isReply"] = (new_esol["parent"]!=0)
mth = new_esol.groupby(by=["Course_day","isReply"]).count()[["discussionid"]]
mth = mth.dropna(axis=0)
mth = mth.reset_index(drop=False)
mth = mth.rename(columns={"discussionid": "Posts Per Day"})

fig, ax = plt.subplots(figsize=(25, 12))
pivot_mth = mth.pivot(index="Course_day", columns="isReply", values="Posts Per Day")
#colors =  plt.cm.tab20b(np.arange(6))
pivot_mth.plot.bar(stacked=True, color=sns.color_palette("colorblind", 48), ax=ax)
ax.set_title("Posts per day by reply status")

In [ ]:
# Basemap UK!
"""
Basemap imports
"""

epsg_loc = file_location + "Continuum/anaconda3/Library/share/"

import os
os.environ["PROJ_LIB"] = epsg_loc #fixr

import matplotlib.cm
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

from matplotlib.lines import Line2D # for the graph legend

# Draw a map with circles at locations in the input dataframe, coloured according to the input palette
# colours are chosen based on values in the input column "colour_col"

# Basemap can also do lines, but this subroutine doesn't

def draw_map(palette,df, colour_col):

    fig, ax = plt.subplots(figsize=(10,20))
    m = Basemap(resolution='i', # c, l, i, h, f or None
                projection='tmerc',
                lat_0=55.3, lon_0=-3.5,
                llcrnrlon=-10.1, llcrnrlat= 52.0, urcrnrlon=2.1, urcrnrlat=59.1)
#m = Basemap(llcrnrlon=-7,llcrnrlat=49,urcrnrlon=2.,urcrnrlat=5.,
#             resolution='i', projection='tmerc', lat_0 = 54, lon_0 = -3.25)
    m.drawmapboundary(fill_color='#46bcec')
    m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')
    m.drawcoastlines()

# Assigning colours to outcomes

    keys = list(df[colour_col].unique())
    cols = sns.color_palette(palette,len(keys)).as_hex()

    colour_dict = {keys[i]: cols[i] for i in range(0,len(keys))}
    
    def plot_circle(start_lon,start_lat,weight,colour):
        m.plot(start_lon, start_lat, 'o', markersize=weight, color=colour, alpha=0.8, latlon=True)
    
    #cutoff=1 # value at which the green turns to red.
    
    for row in df.itertuples(index=True, name='Pandas'):
        #print(row) # for debugging purposes
        lng = getattr(row, "longitude")
        lat = getattr(row, "latitude")
        
        col_definer = getattr(row,colour_col)
        #connector1_status = getattr(row,"connector1Status")
        colour = colour_dict[col_definer]
        wt = 5
            
        if lng!=999 and lat!=999: 
            plot_circle(lng,lat,wt,colour)

     
    #custom_colours = list(colour_dict.values())
    custom_labels = list(colour_dict.keys())
    custom_lines = []
    for key,value in colour_dict.items():
        custom_lines.append(Line2D([0], [0], color=value, lw=4))

    ax.legend(custom_lines, custom_labels)
    ax.set_title('Charging points locations by ' + colour_col, fontsize=16)
